# Module 3: Bilevel Problems


## *From competitive markets equilibrium to stackelberg games*

We consider a simplified diesel market between 3 refineries. Each firm optimizes its own profit, subject only to a production capacity constraint. The firms have different efficiencies, represented by different marginal costs of production. Each player is maximizing their profit (minimising costs minus revenues):

\begin{align}
    \min_{x_i} \ & c_i x_i - p(x) x_i \\
    \text{s.t. } & x_i - X^{max}_i && \leq 0 \quad :\mu_i^{max} \\
    & - x_i && \leq 0 \\
\end{align}

where $p(x)$ is the market price.

In this exercise we investigate **market-price formation** and the firms' **market shares** in different market setups:

1)  Perfect competition  & Welfare maximization

2)  Cournot competition 

3)  Generalized Nash equilibrium 

4)  Stackelberg game  



#### Import code packages

In [1]:
import matplotlib.pyplot
from pyomo.environ import *
import pandas as pd
from pprint import pprint
from IPython.display import display, HTML

#### Problem data

In [2]:
# Marginal costs
c1 = 3
c2 = 4
c3 = 5

# Production capacities
X1_max = 3
X2_max = 4
X3_max = 2

# Demand function parameters
a = 10
b = 1

In [14]:
def market_price(m):
    x = [value(m.x[i]) for i in m.x]
    price = a - b*sum(x)
    return round(price, 2)

def optimal_production(m):
    x = [round(value(m.x[i]),2) for i in m.x]
    return x

def optimal_profits(m):
    profits = []
    p = market_price(m)
    for i in m.x:
        profit = p * value(m.x[i]) - m.c[i] * value(m.x[i])
        profit = max(0, profit) # All profitc are positive in these models, this avoids -0.0 values.
        profits.append(round(profit, 2))
    return profits

def social_welfare(m):
    welfare = a * sum(value(m.x[i]) for i in [1,2,3]) \
                    - b/2 * sum(value(m.x[i]) for i in [1,2,3])**2 \
                  - sum(m.c[i] * value(m.x[i]) for i in [1,2,3])
    return round(welfare,2)

def add_results_to_table(m, setup, table): 
    # Check if setup already exists in the table
    if len(table) == 0 or setup not in table[('Setup', '')].values:
        # Create new row data matching MultiIndex structure
        new_data = {
            ('Setup', ''): [setup],
            ('Market', 'Price'): [market_price(m)],
            ('Production', 'Firm1'): [optimal_production(m)[0]],
            ('Production', 'Firm2'): [optimal_production(m)[1]],
            ('Production', 'Firm3'): [optimal_production(m)[2]],
            ('Profit', 'Firm1'): [optimal_profits(m)[0]],
            ('Profit', 'Firm2'): [optimal_profits(m)[1]],
            ('Profit', 'Firm3'): [optimal_profits(m)[2]],
            ('Social', 'Welfare'): [social_welfare(m)]
        }
        
        new_row = pd.DataFrame(new_data)
        
        if len(table) == 0:
            # If table is empty, just return the new row
            table = new_row
        else:
            # Otherwise, concatenate normally
            table = pd.concat([table, new_row], ignore_index=True)
    return table

def show(table):
    #display(H ML(table.to_html(index=False)))
    # Convert to HTML with custom styling
    html = table.to_html(index=False, table_id="results")
    
    # Add CSS for left alignment
    styled_html = f"""
    <style>
    #results th, #results td {{
        text-align: left !important;
        padding: 6px 12px;
    }}
    #results th {{
        font-weight: bold;
    }}
    #results {{
        border-collapse: collapse;
    }}
    </style>
    {html}
    """
    
    display(HTML(styled_html))
   

In [15]:
columns = pd.MultiIndex.from_tuples([
    ('Setup', ''),
    ('Market', 'Price'),
    ('Production', 'Firm1'),
    ('Production', 'Firm2'),
    ('Production', 'Firm3'),
    ('Profit', 'Firm1'),
    ('Profit', 'Firm2'),
    ('Profit', 'Firm3'), 
    ('Social', 'Welfare')
])
results_table = pd.DataFrame(columns=columns)

show(results_table)

# # 3.1 Competitive market equilibrium & Welfare maximization

### Welfare-maximization model

\begin{align*}
\text{Maximize}_{x_1, x_2, x_3} \quad
& \alpha \sum_{j=1}^{3} x_j
  - \frac{1}{2} \beta \left( \sum_{j=1}^{3} x_j \right)^2
  - \sum_{j=1}^{3} c_j x_j
\\
\text{s.t.} \quad
& x_i - X_i^{\max} \leq 0 : \mu_i^{\max}
    \qquad i = 1,2,3
\\
& -x_i \leq 0,
    \qquad i = 1,2,3.
\end{align*}

In [16]:
# Initialize model for welfare maximization (wm)
wm = ConcreteModel()

# Define parameters
wm.c = Param([1,2,3], initialize={1:c1, 2:c2, 3:c3})
wm.X_max = Param([1,2,3], initialize={1:X1_max, 2:X2_max, 3:X3_max})

# Define nonnegative variables
wm.x = Var([1,2,3], domain=NonNegativeReals)

# Define objective function (social welfare maximization)
wm.obj = Objective(expr = a * sum(wm.x[i] for i in [1,2,3]) \
                    - b/2 * sum(wm.x[i] for i in [1,2,3])**2 \
                  - sum(wm.c[i] * wm.x[i] for i in [1,2,3]), 
                  sense=maximize)

# Define constraints (production capacities)
wm.con1 = Constraint(expr = wm.x[1] <= X1_max)
wm.con2 = Constraint(expr = wm.x[2] <= X2_max)
wm.con3 = Constraint(expr = wm.x[3] <= X3_max)

# Solve the model
solver = SolverFactory('gurobi')
results = solver.solve(wm, tee=False) # tee=False suppresses solver output

print("The solver found an", results.solver.termination_condition, "solution.")

The solver found an optimal solution.


Query results

In [23]:
print("Market price:", market_price(wm))
print("Optimal production (x):", optimal_production(wm))
print("Optimal profits:", optimal_profits(wm))
print("Social welfare:", social_welfare(wm))

Market price: 4.0
Optimal production (x): [3.0, 3.0, 0.0]
Optimal profits: [3.0, 0, 0]
Social welfare: 21.0


In [18]:
results_table = add_results_to_table(wm, 'Welfare Max', results_table)
show(results_table)

### Competitive-market model
Note that this model models a perfect-competition Nash equilibrium.

\begin{align}
& 0 \leq x_i \perp c_i - \alpha + \beta \sum_j x_j + \mu_i^{\max} \geq 0, \quad \forall i \in \{1,2,3\}\\
& 0 \leq \mu_i^{\max} \perp X_i^{\max} - x_i  \geq 0, \quad \forall i \in \{1,2,3\}
\end{align}

In [29]:
# # Initialize competitive market model (cm)
# cm = ConcreteModel()

# # Define parameters
# cm.c = Param([1,2,3], initialize={1:c1, 2:c2, 3:c3})
# cm.X_max = Param([1,2,3], initialize={1:X1_max, 2:X2_max, 3:X3_max})

# # Define variables
# cm.x = Var([1,2,3], domain=NonNegativeReals)
# cm.mu = Var([1,2,3], domain=NonNegativeReals)  # Dual variables

# # Complementarity constraints
# # 0 ≤ x_i ⊥ (c_i - a + b*sum(x_j) + mu_i) ≥ 0
# for i in [1,2,3]:
#     expression = Expression(expr = cm.c[i] - a + b * sum(cm.x[j] for j in [1,2,3]) + cm.mu[i])
#     complementarity(cm, cm.x[i], expression, i)

# # 0 ≤ mu_i ⊥ (X_max - x_i) ≥ 0  
# for i in [1,2,3]:
#     expression = Expression(expr = cm.X_max[i] - cm.x[i])
#     complementarity(cm, cm.mu[i], expression, i)

# # Solve the model
# solver = SolverFactory('gurobi')
# results = solver.solve(wm, tee=False) # tee=False suppresses solver output

# print("The solver found an", results.solver.termination_condition, "solution.")

Query results

In [30]:
# print("Market price:", market_price(cm))
# print("Optimal production (x):", optimal_production(cm))
# print("Optimal profits:", optimal_profits(cm))
# print("Social welfare:", social_welfare(cm))

## The SOS1 variable transformation

Briefly, in the special ordered sets 1 (SOS1) variable transformation, a complementarity constraint of the form $$0 \leq x \perp F(x) \geq 0$$ is reformulated and results in the following constraints:

\begin{align*}
    &2 (v^+ + v^-) = x + F(x) \\
    & 2 (v^+ - v^-) = x - F(x) \\
    & F(x) \geq 0 \\
    & v^+, v^- \text{ are SOS1} \\
    & v^+, v^-, x \geq 0 \\
\end{align*}

In [ ]:
import pyomo.environ as pyo
def complementarity(m:pyo.ConcreteModel, var:pyo.Var, fun:pyo.Expression, i:int):
    """
    Implement a complementarity condition 0 <= x \perp F(x) >= 0 using the SOS1 variable transformation.

    Args:
        m (pyo.ConcreteModel): The Pyomo model to which the constraints and auxiliary varaibles will be added to.
        var (pyo.Var): The variable that the complementarity condition applies to.
        fun (pyo.Expression): The expression that defines the complementarity condition.
    Returns:
        None: The function modifies the model in place.
    """

    # Adding SOS1 variables to model
    m.add_component('v_plus_' + var.name, pyo.Var(domain=pyo.NonNegativeReals))
    m.add_component('v_minus_' + var.name, pyo.Var(domain=pyo.NonNegativeReals))

    # Creating python variables pointin to model variables for easy access
    v_plus = m.component('v_plus_' + var.name)
    v_minus = m.component('v_minus_' + var.name)
    
    # Add constraints to the model
    m.add_component('SOS1_' + var.name + '_nonnegativity', 
                   pyo.Constraint(expr = fun >= 0))
    
    m.add_component('SOS1_' + var.name + '_addition', 
                   pyo.Constraint(expr = 2*(v_plus + v_minus) == var + fun))
    
    m.add_component('SOS1_' + var.name + '_subtraction', 
                   pyo.Constraint(expr = 2*(v_plus - v_minus) == var - fun))
    
    # Create function for generating SOS1 constraints
    def SOS1_rule(m):
        vars_list = [v_plus, v_minus]
        weights = [1, 2] # mandatory to specify weights, these may affect computation performance but for now set naively. 
        return vars_list, weights
    
    m.add_component('SOS1_' + var.name + '_constraint',
                    pyo.SOSConstraint(rule=SOS1_rule, sos=1))